In [14]:
import requests
from datetime import date, datetime, timedelta
import time
import os
import glob
import logging
import akvut
import pyodbc
from io import BytesIO
import gzip
import pandas as pd


idag = date.today().isoformat()
logger = akvut.lag_logger(f'loggfil-les_web_logs-{idag}.log')



In [2]:
  access_token = akvut.les_access_token(logger)

2025-01-21 13:13:54,614 - my_logger - INFO - Henta access_token OK.


In [3]:
sist_oppdatert = (datetime.now() - timedelta(days=1)).isoformat() + "Z"

In [4]:
start_sjekk_status = time.perf_counter()
requesturl = "https://api-gateway.instructure.com/dap/query/canvas_logs/table/web_logs/data"
payload = '{"format": "csv", "since": \"%s\"}' % (sist_oppdatert)
headers = {'x-instauth': access_token, 'Content-Type': 'text/plain'}
logger.info(f"Sender (inkrementel) spørjing til {requesturl}")
r = requests.request(
    "POST",
    requesturl,
    headers=headers,
    data=payload
)
if r.status_code == 200:
    respons = r.json()
    id = respons['id']
    vent = True
    while vent:
        logger.info(f"Sjekker status på jobb {id}")
        requesturl = f"https://api-gateway.instructure.com/dap//job/{id}"
        r2 = requests.request("GET", requesturl, headers=headers)
        respons2 = r2.json()
        if respons2['status'] == "complete":
            vent = False
        time.sleep(5)
    logger.info("Status oppdatert")
else:
    logger.error(f"Feil i spørjing, kode {r.status_code}")
antal = len(respons2['objects'])
tidsbruk_sjekk_status = time.perf_counter() - start_sjekk_status
logging.info(f"Totalt for sjekk_status: {tidsbruk_sjekk_status}")

2025-01-21 13:14:01,607 - my_logger - INFO - Sender (inkrementel) spørjing til https://api-gateway.instructure.com/dap/query/canvas_logs/table/web_logs/data
2025-01-21 13:14:02,335 - my_logger - INFO - Sjekker status på jobb 9589bc84-7439-43d4-8743-4e9af0a9e95d
2025-01-21 13:14:07,839 - my_logger - INFO - Sjekker status på jobb 9589bc84-7439-43d4-8743-4e9af0a9e95d
2025-01-21 13:14:13,741 - my_logger - INFO - Sjekker status på jobb 9589bc84-7439-43d4-8743-4e9af0a9e95d
2025-01-21 13:14:19,172 - my_logger - INFO - Sjekker status på jobb 9589bc84-7439-43d4-8743-4e9af0a9e95d
2025-01-21 13:14:24,969 - my_logger - INFO - Sjekker status på jobb 9589bc84-7439-43d4-8743-4e9af0a9e95d
2025-01-21 13:14:30,410 - my_logger - INFO - Sjekker status på jobb 9589bc84-7439-43d4-8743-4e9af0a9e95d
2025-01-21 13:14:35,853 - my_logger - INFO - Sjekker status på jobb 9589bc84-7439-43d4-8743-4e9af0a9e95d
2025-01-21 13:14:41,287 - my_logger - INFO - Sjekker status på jobb 9589bc84-7439-43d4-8743-4e9af0a9e95d
202

In [5]:
start_hent_filar = time.perf_counter()
web_logs_i_dag = []

In [6]:
innfil = respons2['objects'][0]['id']
payload = f"{respons2['objects']}"

In [7]:
requesturl = "https://api-gateway.instructure.com/dap/object/url"
payload = payload.replace('\'', '\"')
headers = {
    'x-instauth': access_token,
    'Content-Type': 'text/plain'
}
    

In [8]:
r4 = requests.request(
    "POST",
    requesturl,
    headers=headers,
    data=payload
)

In [9]:
respons4 = r4.json()
url = respons4['urls'][innfil]['url']
data = requests.request("GET", url)


In [15]:
with gzip.open(BytesIO(data.content), 'rb') as f:
    # Assuming the file is a CSV inside the .gz
    df = pd.read_csv(f)